In [ ]:
pip install langchain langchain_openai langchain_community chromadb

In [ ]:
from google.colab import drive
import os

# 먼저 구글 드라이브 마운트
drive.mount('/content/drive')

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("")

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 문서 로더 설정
loaders = [TextLoader("./Data/How_to_invest_money.txt")]

docs = []
for loader in loaders:
    docs.extend(loader.load())


In [4]:
# 문서 생성을 위한 텍스트 분할기 정의
recursive_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# 문서 분할
split_docs = recursive_splitter.split_documents(docs)

# OpenAIEmbeddings 인스턴스 생성
embeddings = OpenAIEmbeddings()

# Chroma vectorstore 생성
vectorstore = Chroma.from_documents(documents=split_docs, embedding=embeddings)

# Chroma vectorstore 기반 리트리버 생성
retriever = vectorstore.as_retriever()

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_text_splitters import CharacterTextSplitter


# 1. 가상 문서 생성 체인
def create_virtual_doc_chain():
    system = "당신은 고도로 숙련된 AI입니다."
    user = """
    주어진 질문 '{query}'에 대해 직접적으로 답변하는 가상의 문서를 생성하세요.
    문서의 크기는 {chunk_size} 글자 언저리여야 합니다.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", user)
    ])
    llm = ChatOpenAI(model="gpt-4o", temperature=0.2)
    return prompt | llm | StrOutputParser()

In [6]:
# 2. 문서 검색 체인
def create_retrieval_chain():
    return RunnableLambda(lambda x: retriever.get_relevant_documents(x['virtual_doc']))

# 유틸리티 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [7]:
# 3. 최종 응답 생성 체인
def create_final_response_chain():
    final_prompt = ChatPromptTemplate.from_template("""
    다음 정보와 질문을 바탕으로 답변해주세요:

    컨텍스트: {context}

    질문: {question}

    답변:
    """)
    final_llm = ChatOpenAI(model="gpt-4o", temperature=0.2)
    return final_prompt | final_llm

In [8]:
from langchain_core.runnables import RunnableLambda

def print_input_output(input_data, output_data, step_name):
    print(f"\n--- {step_name} ---")
    print(f"Input: {input_data}")
    print(f"Output: {output_data}")
    print("-" * 50)

In [9]:
def create_pipeline_with_logging():
    virtual_doc_chain = create_virtual_doc_chain()
    retrieval_chain = create_retrieval_chain()
    final_response_chain = create_final_response_chain()

    # 가상 문서 생성 단계
    def virtual_doc_step(x):
        result = {"virtual_doc": virtual_doc_chain.invoke({
            "query": x["question"],
            "chunk_size": 200
        })}
        print_input_output(x, result, "Virtual Doc Generation")
        return {**x, **result}

    # 문서 검색 단계
    def retrieval_step(x):
        result = {"retrieved_docs": retrieval_chain.invoke(x)}
        print_input_output(x, result, "Document Retrieval")
        return {**x, **result}

    # 컨텍스트 포맷팅 단계
    def context_formatting_step(x):
        result = {"context": format_docs(x["retrieved_docs"])}
        print_input_output(x, result, "Context Formatting")
        return {**x, **result}

    # 최종 응답 생성 단계
    def final_response_step(x):
        result = final_response_chain.invoke(x)
        print_input_output(x, result, "Final Response Generation")
        return result

    # 전체 파이프라인 구성
    pipeline = (
        RunnableLambda(virtual_doc_step)
        | RunnableLambda(retrieval_step)
        | RunnableLambda(context_formatting_step)
        | RunnableLambda(final_response_step)
    )

    return pipeline

# 파이프라인 객체 생성
pipeline = create_pipeline_with_logging()

In [10]:
# 예시 질문과 답변
question = "주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?"
response = pipeline.invoke({"question": question})
print(f"최종 답변: {response.content}")


--- Virtual Doc Generation ---
Input: {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?'}
Output: {'virtual_doc': '주식 시장의 변동성이 높을 때 투자 전략은 신중한 접근이 필요합니다. 첫째, 포트폴리오를 다각화하여 리스크를 분산시키는 것이 중요합니다. 둘째, 방어주나 배당주와 같은 안정적인 주식에 투자하여 수익을 확보할 수 있습니다. 셋째, 현금을 보유하여 기회를 기다리는 것도 전략 중 하나입니다. 마지막으로, 시장의 변동성을 활용하여 단기 트레이딩을 고려할 수 있지만, 이는 높은 리스크를 동반하므로 주의가 필요합니다. 전문가의 조언을 참고하여 전략을 세우는 것이 바람직합니다.'}
--------------------------------------------------


C:\Users\User\AppData\Local\Temp\ipykernel_23504\1353087294.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return RunnableLambda(lambda x: retriever.get_relevant_documents(x['virtual_doc']))



--- Document Retrieval ---
Input: {'question': '주식 시장의 변동성이 높을 때 투자 전략은 무엇인가요?', 'virtual_doc': '주식 시장의 변동성이 높을 때 투자 전략은 신중한 접근이 필요합니다. 첫째, 포트폴리오를 다각화하여 리스크를 분산시키는 것이 중요합니다. 둘째, 방어주나 배당주와 같은 안정적인 주식에 투자하여 수익을 확보할 수 있습니다. 셋째, 현금을 보유하여 기회를 기다리는 것도 전략 중 하나입니다. 마지막으로, 시장의 변동성을 활용하여 단기 트레이딩을 고려할 수 있지만, 이는 높은 리스크를 동반하므로 주의가 필요합니다. 전문가의 조언을 참고하여 전략을 세우는 것이 바람직합니다.'}
Output: {'retrieved_docs': [Document(metadata={'source': './Data/How_to_invest_money.txt'}, page_content='After learning how to judge the value of every form of investment, a man\nmay still be unsuccessful in the investment of money unless he acquires\nalso a firm grasp upon the general principles which control the price\nmovements of securities. By this it is not meant that a man needs to\nhave an intimate knowledge of technical market conditions whereby to\nestimate temporary fluctations of minor importance, but rather that he\nshould have clearly in mind the causes which operate to produce the\nlarger swings of prices. If an i